## Chap05-1. Model Save
### 모델을 저장하고 재사용하는 방법 익히기

In [5]:
import tensorflow as tf
import numpy as np

In [6]:
# 1) 학습데이터 불러오기

data = np.loadtxt('./data.csv', delimiter=',',
                 unpack=True, dtype='float32', skiprows=0)

# 털, 날개, 기타, 포유류, 조류
#  - x_data = 0, 1 (털, 날개)
#  - y_data = 2, 3, 4 (기타, 포유류, 조류)
x_data = np.transpose(data[0:2])
y_data = np.transpose(data[2:])

In [7]:
# 2) 신경망 모델 구성

# glbal_step: 학습 횟수 카운트 변수 설정
#  - 학습에 직접 사용되지 않고, 학습 횟수를 카운트하는 변수
global_step = tf.Variable(0, trainable=False, name='global_step')

# Input(X), Output(Y) 설정
X = tf.placeholder(tf.float32)
Y = tf.placeholder(tf.float32)

# input -> hidden(1)[10] -> hidden(2)[20] -> output
# Parameter 설정: W만 설정 b는 생략
W1 = tf.Variable(tf.random_uniform([2, 10], -1., 1.))
L1 = tf.nn.relu(tf.matmul(X, W1))

W2 = tf.Variable(tf.random_uniform([10, 20], -1., 1.))
L2 = tf.nn.relu(tf.matmul(L1, W2))

W3 = tf.Variable(tf.random_uniform([20, 3], -1., 1.))
model = tf.matmul(L2, W3)

# Loss Function: Cross-Entropy
cost = tf.losses.softmax_cross_entropy(onehot_labels=Y, logits=model)

# Optimizer: Adam
optimizer = tf.train.AdamOptimizer(learning_rate=0.01)

# global_step으로 넘겨준 변수를, 파라미터(W1, W2, W3)들을 최적화 할 때 마다 학습 획수를 하나씩 증가
train_op = optimizer.minimize(cost, global_step=global_step)

In [10]:
# 3) 신경망 모델 학습
sess = tf.Session()

# 모델을 저장하고 불러오는 API를 초기화
#  - tf.global_variables() : 앞에서 정의한 변수들(tf.Variable) 가져오는 함수
#                            이 변수들을 파일에 저장하거나 불러올 변수들로 사용함
saver = tf.train.Saver(tf.global_variables())

# ./model 디렉터리에 기존에 학습해둔 모델이 있는지 확인 후
# - 모델이 있다면 saver.restore()를 사용해 학습된 값들을 불러오고,
# - 모델이 없다면 변수를 새로 초기화함
# - 학습된 모델을 저장한 파일을 체크포인트 파일(checkpoint file)이라고 함
ckpt = tf.train.get_checkpoint_state('./model')
if ckpt and tf.train.checkpoint_exists(ckpt.model_checkpoint_path):
    saver.restore(sess, ckpt.model_checkpoint_path)
else:
    sess.run(tf.global_variables_initializer())
    
# 최적화 진행
for step in range(10):
    sess.run(train_op, feed_dict={X: x_data, Y: y_data})
    
    print('Step: {:d}  Cost: {:.3f}'.format(sess.run(global_step), 
                                           sess.run(cost, feed_dict={X: x_data, Y: y_data})))
    
# 최적화가 끝난 뒤, 변수를 저장
saver.save(sess, './model/dnn.ckpt', global_step=global_step)

INFO:tensorflow:Restoring parameters from ./model/dnn.ckpt-10
Step: 11  Cost: 0.658
Step: 12  Cost: 0.643
Step: 13  Cost: 0.630
Step: 14  Cost: 0.619
Step: 15  Cost: 0.610
Step: 16  Cost: 0.602
Step: 17  Cost: 0.596
Step: 18  Cost: 0.590
Step: 19  Cost: 0.586
Step: 20  Cost: 0.581


'./model/dnn.ckpt-20'

In [11]:
# 4) 결과 확인
#  - 0:기타, 1: 포유류, 2: 조류

prediction = tf.argmax(model, 1)
target = tf.argmax(Y, 1)
print('예측값:', sess.run(prediction, feed_dict={X: x_data}))
print('실제값:', sess.run(target, feed_dict={Y: y_data}))

is_correct = tf.equal(prediction, target)
accuracy = tf.reduce_mean(tf.cast(is_correct, tf.float32))
print('정확도: %.2f' % sess.run(accuracy * 100, feed_dict={X: x_data, Y: y_data}))

예측값: [0 1 2 0 0 2]
실제값: [0 1 2 0 0 2]
정확도: 100.00
